# Paramétrage de l'environnement de travail et import des packages

In [205]:
%cd "C:/Users/user/Desktop/data_science_documents/data_science_projets_perso/projet_housing_prediction/housing-prediction"

C:\Users\user\Desktop\data_science_documents\data_science_projets_perso\projet_housing_prediction\housing-prediction


In [206]:
from pathlib import Path

%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

ROOT = Path().resolve()
sys.path.append(str(ROOT / "03_src"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [213]:
import pandas as pd
import numpy as np
import duckdb
from econometrics.cleaning_ante_econometrics import nettoyage_post_eda


In [208]:
chemin_repo = Path(r"C:\Users\user\Desktop\data_science_documents\data_science_projets_perso\projet_housing_prediction\housing-prediction")

# Base de données avant nettoyage

**Import de la base de données**

In [209]:
data_housing = pd.read_csv(f"{chemin_repo}/01_data/02_processed/01_merge/housing_open_data_processed.csv",
                           dtype={'id_annonce':"object"})

colonnes_retenus = ["id_annonce","code_insee",
                    "approximate_latitude","approximate_longitude",
                    "size","nb_rooms","nb_photos","population",
                    "nb_menages_fiscaux","revenu_median","nb_actes_delinquants",
                    "nb_lits_tourisme","densite_services_rayon",
                    "type_rural_urbain","position_commune_unite_urbaine",
                    "property_type","type_degre_densite","type_unite_urbaine",
                    "taille_pole_et_couronne","taille_agglomeration",
                    "target"]

data_housing = data_housing[colonnes_retenus]

**Statistiques descriptives globales**

In [210]:
data_housing.describe()

,approximate_latitude,approximate_longitude,size,nb_rooms,nb_photos,population,nb_menages_fiscaux,revenu_median,nb_actes_delinquants,nb_lits_tourisme,densite_services_rayon,target
count,29894.000000,29894.000000,29476.000000,28664.000000,29894.000000,29671.000000,29522.000000,29522.000000,29644.000000,29644.000000,29894.000000,29894.000000
mean,46.547615,2.616198,1093.863380,4.247279,7.989697,44026.389134,19836.333209,24110.408170,3567.963095,591.877952,151.100154,0.326755
std,2.353455,2.593866,5779.710007,2.982144,4.649502,81315.177423,36456.868986,5132.887082,7265.176921,1439.118121,260.652977,0.469035
min,41.374436,-4.718490,1.000000,0.000000,1.000000,14.000000,41.000000,13460.000000,0.000000,0.000000,0.000000,0.000000
25%,43.928447,1.095563,74.000000,3.000000,5.000000,2367.000000,1073.000000,20960.000000,50.000000,0.000000,12.000000,0.000000
50%,46.933041,2.378032,115.000000,4.000000,8.000000,11053.000000,5160.000000,22800.000000,574.000000,0.000000,44.000000,0.000000
75%,48.841204,4.620722,237.000000,5.000000,10.000000,47160.000000,21373.000000,25750.000000,3348.000000,522.000000,158.000000,1.000000
max,51.075892,9.503198,411311.000000,125.000000,50.000000,504078.000000,222023.000000,55340.000000,43378.000000,16734.000000,1524.000000,1.000000


**Statistiques descriptives sur 'nb_rooms' par modalité de la variable 'property_type'**

In [211]:
stats_nbrooms = (data_housing
 .groupby(["property_type"])["nb_rooms"]
 .agg(
     min_nbrooms=lambda x: x.min(),
     q1_nbrooms=lambda x: x.quantile(0.25),
     mediane_nbrooms=lambda x: x.median(),
     moyenne_nbrooms=lambda x: x.mean(),
     q3_nbrooms=lambda x: x.quantile(0.75),
     max_nbrooms=lambda x: x.max()
 )
).reset_index()

stats_nbrooms

,property_type,min_nbrooms,q1_nbrooms,mediane_nbrooms,moyenne_nbrooms,q3_nbrooms,max_nbrooms
0,appartement,1.0,2.00,3.0,3.028883,4.00,28.0
1,atelier,3.0,3.50,4.0,4.000000,4.50,5.0
2,chalet,2.0,3.00,4.0,4.578947,6.00,8.0
3,chambre,1.0,1.00,1.0,1.000000,1.00,1.0
4,château,5.0,12.00,15.0,14.647059,20.00,27.0
5,divers,1.0,3.00,5.0,7.657377,10.00,118.0
6,duplex,1.0,3.00,4.0,3.959302,5.00,10.0
7,ferme,1.0,5.00,6.0,6.555556,8.00,19.0
8,gîte,7.0,7.50,8.0,10.000000,11.50,15.0
9,hôtel,NaN,NaN,NaN,NaN,NaN,NaN


**Statistiques descriptives sur 'size' par modalité de la variable 'property_type'**

In [212]:
stats_size = (data_housing
 .groupby(["property_type"])["size"]
 .agg(
     min_size=lambda x: x.min(),
     q1_size=lambda x: x.quantile(0.25),
     mediane_size=lambda x: x.median(),
     moyenne_size=lambda x: x.mean(),
     q3_size=lambda x: x.quantile(0.75),
     max_size=lambda x: x.max()
 )
).reset_index()

stats_size


,property_type,min_size,q1_size,mediane_size,moyenne_size,q3_size,max_size
0,appartement,3.0,55.00,78.0,1220.120501,185.00,23539.0
1,atelier,79.0,82.25,85.5,85.500000,88.75,92.0
2,chalet,30.0,61.00,120.0,321.921053,174.50,7353.0
3,chambre,10.0,547.75,795.0,684.000000,931.25,1136.0
4,château,150.0,440.00,600.0,617.058824,700.00,1420.0
5,divers,1.0,80.00,150.0,964.270199,308.75,250000.0
6,duplex,25.0,74.50,102.0,1689.649123,169.50,14667.0
7,ferme,40.0,128.75,167.5,3950.703704,241.25,400000.0
8,gîte,190.0,215.00,240.0,253.333333,285.00,330.0
9,hôtel,1600.0,1600.00,1600.0,1600.000000,1600.00,1600.0


# Base de données après nettoyage

**Nettoyage des données**

In [219]:
data_housing = nettoyage_post_eda()

Import de la base de données : OK
Selection des colonnes d'interet : OK
Statistiques descriptives pour 'size' et 'nb_rooms' par modalité de 'property_type' : OK
Passage de viager en appartement : OK
Requete SQL : OK
Renommage des colonnes 'size' et 'nb_rooms' : OK
Passage à 3 modalités sur 'property_type' (appartement, maison, autre)' : OK
✅ Datasource 'housing_project' prête.
📸 Asset 'housing_post_merge' mis à jour.
(Great Exepectations) Paramétrage du batch : OK


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

(Great Exepectations) Paramétrage des conditions : OK


Calculating Metrics:   0%|          | 0/33 [00:00<?, ?it/s]

Règle 1 (expect_column_stdev_to_be_between): OK
Règle 2 (expect_column_stdev_to_be_between): OK
Règle 3 (expect_column_values_to_be_in_set): OK
Règle 4 (expect_compound_columns_to_be_unique): OK
Règle 5 (expect_table_column_count_to_equal): OK
Règle 6 (expect_column_to_exist): OK
Règle 7 (expect_column_values_to_not_be_null): OK
Règle 8 (expect_column_value_lengths_to_be_between): OK
Règle 9 (expect_column_value_lengths_to_be_between): OK
Règle 10 (expect_column_values_to_be_unique): OK
Check de la validité des données : OK
Export de la table : OK
Fin de la pipeline !


**Statistiques descriptives globales**

In [221]:
data_housing.describe()

,approximate_latitude,approximate_longitude,size,nb_rooms,nb_photos,population,nb_menages_fiscaux,revenu_median,nb_actes_delinquants,nb_lits_tourisme,densite_services_rayon,target
count,26648.000000,26648.000000,26479.000000,26505.000000,26648.000000,26446.000000,26317.000000,26317.000000,26422.000000,26422.000000,26648.000000,26648.00000
mean,46.560064,2.653694,124.804439,4.380117,8.470129,45361.397035,20416.650264,24194.397158,3640.282643,612.029937,152.181852,0.34618
std,2.358976,2.583976,71.485633,2.212467,4.505669,81953.737109,36761.758055,5199.124632,7281.805559,1453.082904,256.366925,0.47576
min,41.374436,-4.718490,10.000000,1.000000,1.000000,14.000000,41.000000,13460.000000,0.000000,0.000000,0.000000,0.00000
25%,43.896935,1.198781,72.000000,3.000000,5.000000,2613.000000,1200.000000,20970.000000,62.000000,0.000000,12.000000,0.00000
50%,46.985218,2.385479,110.000000,4.000000,8.000000,12096.000000,5562.000000,22870.000000,654.000000,0.000000,47.000000,0.00000
75%,48.844464,4.683312,185.000000,5.000000,10.000000,49104.000000,22023.000000,25810.000000,3482.000000,557.000000,164.000000,1.00000
max,51.046009,9.503198,1600.000000,27.000000,50.000000,504078.000000,222023.000000,55340.000000,43378.000000,16734.000000,1524.000000,1.00000


Ce notebook transforme les données immobilières brutes en un dataset propre pour la modélisation. L'objectif est de corriger les erreurs physiques (surfaces, nombre de pièces) sans supprimer massivement des données, tout en garantissant la qualité du résultat final via des tests automatisés.

Démarche
1. Nettoyage contextuel par type de bien
Plutôt que d'appliquer des filtres globaux qui supprimeraient des biens atypiques, j'ai calculé des statistiques (quartiles, min, max) spécifiques à chaque property_type. Cela permet de définir des bornes de surface cohérentes pour chaque catégorie (ex: 80-300m² pour une villa, 10-200m² pour un appartement).

2. Imputation conditionnelle via SQL/DuckDB
J'ai utilisé DuckDB pour exécuter des règles d'imputation complexes basées sur la cohérence entre surface et nombre de pièces.

Logique exacte : Si une surface est hors bornes (ex: villa > 300m²), je vérifie le nombre de pièces.

Si le nombre de pièces est lui aussi très élevé (> Q3), je ramène la surface à la borne max (300m²).

Sinon, je considère la surface comme erronée et je l'impute par le 3ème quartile statistique du type de bien.

Même logique symétrique pour les bornes inférieures.

3. Feature Engineering
Pour simplifier la modélisation future, j'ai recodé la variable property_type en regroupant les catégories rares sous le label "autre", tout en gardant les catégories principales "appartement" et "maison".

4. Validation avec Great Expectations
J'ai intégré un verrou de qualité en fin de pipeline. L'export du dataset n'est validé que si les données passent une série de tests définis avec Great Expectations (cohérence surface/pièces, unicité des ID, pas de valeurs nulles sur la target, etc.).

Stack technique
Pandas : Manipulation des DataFrames.

DuckDB : Traitements SQL performants et imputations conditionnelles avancées.

Great Expectations : Validation automatique de la qualité des données.